In [1]:
import xport

import numpy as np
import scipy as scp

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# sns.set(color_codes=True)
%matplotlib inline

In [2]:
# open sleep dataset
with open("../../../data/nhanes/2011_2012/SLQ_G.XPT", 'rb') as f:
    sleep_df = xport.to_dataframe(f)

# open alcohol dataset
with open("../../../data/nhanes/2011_2012/ALQ_G.XPT", 'rb') as f:
    alc_df = xport.to_dataframe(f)

# open body measure dataset
with open("../../../data/nhanes/2011_2012/BMX_G.XPT", 'rb') as f:
    body_metric_df = xport.to_dataframe(f)

# see all the output for a little bit
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

print(sleep_df.shape)
print(alc_df.shape)
print(body_metric_df.shape)

sleep_df.head()
alc_df.head()
body_metric_df.head()

(6175, 4)
(5615, 10)
(9338, 26)


,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXSAD1,BMXSAD2,BMXSAD3,BMXSAD4,BMDAVSAD,BMDSADCM
0,62161.0,1.0,69.2,NaN,NaN,NaN,NaN,NaN,172.3,NaN,...,32.5,NaN,81.0,NaN,17.7,17.9,NaN,NaN,17.8,NaN
1,62162.0,1.0,12.7,NaN,95.7,NaN,NaN,NaN,94.7,NaN,...,16.6,NaN,45.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62163.0,1.0,49.4,NaN,NaN,NaN,NaN,NaN,168.9,NaN,...,22.0,NaN,64.6,NaN,15.6,15.5,NaN,NaN,15.6,NaN
3,62164.0,1.0,67.2,NaN,NaN,NaN,NaN,NaN,170.1,NaN,...,29.3,NaN,80.1,NaN,18.3,18.5,NaN,NaN,18.4,NaN
4,62165.0,1.0,69.1,NaN,NaN,NaN,NaN,NaN,159.4,NaN,...,29.7,NaN,86.7,NaN,21.0,20.8,NaN,NaN,20.9,NaN


for sleep: SLQ050 - Ever told doctor had trouble sleeping?

In [3]:
# now print last
InteractiveShell.ast_node_interactivity = "all"

# remove all idk, missing, etc.
clean_alc_df = alc_df.loc[alc_df.ALQ151 < 7]
# print(clean_alc_df.shape)
clean_alc_col_list = ['SEQN','ALQ151']
clean_alc_df = clean_alc_df[clean_alc_col_list].dropna()

clean_sleep_col_list = ['SEQN','SLQ050']
clean_sleep_df = sleep_df.loc[sleep_df.SLQ050 < 9]
clean_sleep_df = clean_sleep_df[clean_sleep_col_list].dropna()

clean_bmi_col_list = ['SEQN','BMXBMI']
clean_bmi_df = body_metric_df
clean_bmi_df = clean_bmi_df[clean_bmi_col_list].dropna()


In [4]:
combined_dataset = pd.merge(clean_alc_df, clean_sleep_df, on="SEQN")
combined_dataset = pd.merge(combined_dataset, clean_bmi_df, on="SEQN")
print(combined_dataset.shape)

total_responders = combined_dataset.shape[0]

(4130, 4)


In [5]:
alc1_sleep1_obesity1_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)])
alc1_sleep1_obesity0_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)])
alc1_sleep0_obesity1_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)])
alc1_sleep0_obesity0_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)])
print(alc1_sleep1_obesity1_og_pop_size)
print(alc1_sleep1_obesity0_og_pop_size)
print(alc1_sleep0_obesity1_og_pop_size)
print(alc1_sleep0_obesity0_og_pop_size)

print("\n")
alc0_sleep1_obesity1_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)])
alc0_sleep1_obesity0_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)])
alc0_sleep0_obesity1_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)])
alc0_sleep0_obesity0_og_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)])
print(alc0_sleep1_obesity1_og_pop_size)
print(alc0_sleep1_obesity0_og_pop_size)
print(alc0_sleep0_obesity1_og_pop_size)
print(alc0_sleep0_obesity0_og_pop_size)


185
43
309
107


626
187
1920
753


We will now start some IP weighting. Yayyy!  

alc is confounder  -  ALQ151 yes(1), 2 (no)  
sleep is treatment  - SLQ050  yes(1), 2 (no)  
obesity is outcome  - BMXBMI anything above 24 is overweight  

In [6]:
# conditional probability pop sizes
alc1_pop_size = len(combined_dataset[combined_dataset.ALQ151 == 1])
alc0_pop_size = len(combined_dataset[combined_dataset.ALQ151 == 2])
# print(alc1_pop_size)
# print(alc0_pop_size)
# print(len(combined_dataset))
# first layer
alc1_prob = len(combined_dataset[combined_dataset.ALQ151 == 1]) / total_responders
alc0_prob = len(combined_dataset[combined_dataset.ALQ151 == 2]) / total_responders
# print(alc1_prob)
# print(alc0_prob)
# print("\n")
# second layer

#conditional probability pop sizes
alc1_sleep1_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1)])
alc1_sleep0_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2)])
alc0_sleep1_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1)])
alc0_sleep0_pop_size = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2)])

# print(alc1_sleep1_pop_size)
# print(alc1_sleep0_pop_size)
# print(alc0_sleep1_pop_size)
# print(alc0_sleep0_pop_size)

alc1_sleep1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1)]) / alc1_pop_size
alc1_sleep0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2)]) / alc1_pop_size
alc0_sleep1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1)]) / alc0_pop_size
alc0_sleep0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2)]) / alc0_pop_size
# print(alc1_sleep1_prob)
# print(alc1_sleep0_prob)
# print(alc0_sleep1_prob)
# print(alc0_sleep0_prob)
# print("\n")

# third layer
alc1_sleep1_obesity1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)]) / alc1_sleep1_pop_size
alc1_sleep1_obesity0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)]) / alc1_sleep1_pop_size
alc1_sleep0_obesity1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)]) / alc1_sleep0_pop_size
alc1_sleep0_obesity0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)]) / alc1_sleep0_pop_size
print(alc1_sleep1_obesity1_prob)
print(alc1_sleep1_obesity0_prob)
print(alc1_sleep0_obesity1_prob)
print(alc1_sleep0_obesity0_prob)

print("\n")
alc0_sleep1_obesity1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)]) / alc0_sleep1_pop_size
alc0_sleep1_obesity0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)]) / alc0_sleep1_pop_size
alc0_sleep0_obesity1_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)]) / alc0_sleep0_pop_size
alc0_sleep0_obesity0_prob = len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)]) / alc0_sleep0_pop_size
print(alc0_sleep1_obesity1_prob)
print(alc0_sleep1_obesity0_prob)
print(alc0_sleep0_obesity1_prob)
print(alc0_sleep0_obesity0_prob)
print("\n")

0.8114035087719298
0.18859649122807018
0.7427884615384616
0.25721153846153844


0.7699876998769988
0.23001230012300122
0.7182940516273849
0.28170594837261503




In [7]:
weight_sleep0_given_alc0 = 1.0/alc0_sleep0_prob
weight_sleep1_given_alc0 = 1.0/alc0_sleep1_prob
weight_sleep0_given_alc1 = 1.0/alc1_sleep0_prob
weight_sleep1_given_alc1 = 1.0/alc1_sleep1_prob
print(weight_sleep0_given_alc0)
print(weight_sleep0_given_alc1)
print(weight_sleep1_given_alc0)
print(weight_sleep1_given_alc1)

1.3041526374859707
1.5480769230769231
4.287822878228782
2.824561403508772


In [8]:
alc0_sleep0_obesity0_pop_size = weight_sleep0_given_alc0 * len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)])
alc0_sleep0_obesity1_pop_size = weight_sleep0_given_alc0 * len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)])

print(alc0_sleep0_obesity0_pop_size)
print(alc0_sleep0_obesity1_pop_size)
print("\n")

alc0_sleep1_obesity0_pop_size = weight_sleep1_given_alc0 * len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)])
alc0_sleep1_obesity1_pop_size = weight_sleep1_given_alc0 * len(combined_dataset[(combined_dataset.ALQ151 == 2) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)])

print(alc0_sleep1_obesity0_pop_size)
print(alc0_sleep1_obesity1_pop_size)
print("\n")

alc1_sleep0_obesity0_pop_size = weight_sleep0_given_alc1 * len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI <= 24.0)])
alc1_sleep0_obesity1_pop_size = weight_sleep0_given_alc1 * len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 2) & (combined_dataset.BMXBMI > 24.0)])

print(alc1_sleep0_obesity0_pop_size)
print(alc1_sleep0_obesity1_pop_size)
print("\n")

alc1_sleep1_obesity0_pop_size = weight_sleep1_given_alc1 * len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI <= 24.0)])
alc1_sleep1_obesity1_pop_size = weight_sleep1_given_alc1 * len(combined_dataset[(combined_dataset.ALQ151 == 1) & (combined_dataset.SLQ050 == 1) & (combined_dataset.BMXBMI > 24.0)])

print(alc1_sleep1_obesity0_pop_size)
print(alc1_sleep1_obesity1_pop_size)
print("\n")

print(alc0_sleep0_obesity0_pop_size + alc0_sleep0_obesity1_pop_size + alc0_sleep1_obesity0_pop_size + alc0_sleep1_obesity1_pop_size + alc1_sleep0_obesity0_pop_size + alc1_sleep0_obesity1_pop_size + alc1_sleep1_obesity0_pop_size + alc1_sleep1_obesity1_pop_size)

pseudo_pop = total_responders * 2
print(pseudo_pop)

982.0269360269359
2503.9730639730637


801.8228782287823
2684.1771217712176


165.64423076923077
478.3557692307692


121.45614035087719
522.5438596491227


8259.999999999998
8260


In [25]:
import math
# print(alc0_sleep0_obesity0_pop_size + alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size + alc1_sleep0_obesity0_pop_size)
# print(alc0_sleep1_obesity0_pop_size + alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity0_pop_size + alc1_sleep1_obesity1_pop_size)
# print(pseudo_pop)
untreat_pop_size = alc0_sleep0_obesity0_pop_size + alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size + alc1_sleep0_obesity0_pop_size
treated_pop_size = alc0_sleep1_obesity0_pop_size + alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity0_pop_size + alc1_sleep1_obesity1_pop_size


causal_diff = ((alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity1_pop_size)/treated_pop_size) - ((alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size)/untreat_pop_size)
print("Causal Risk Difference: %f" % causal_diff)

causal_diff_part1 = (alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity1_pop_size)/treated_pop_size
causal_diff_part2 = (alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size)/untreat_pop_size


uncertain1 = math.sqrt((causal_diff_part1 * (1-causal_diff_part1))/(treated_pop_size))
uncertain2 = math.sqrt((causal_diff_part2 * (1-causal_diff_part2))/(untreat_pop_size))
print("uncertain1", 1.96 * math.sqrt(math.pow(uncertain1,2) + math.pow(uncertain2,2)))


print('\n')

causal_risk_ratio = ((alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity1_pop_size)/treated_pop_size) / ((alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size)/untreat_pop_size)
print("Causal Risk Ratio: %f" % causal_risk_ratio)


causal_ratio_part1 = (alc0_sleep1_obesity1_pop_size + alc1_sleep1_obesity1_pop_size)/treated_pop_size
causal_ratio_part2 = (alc0_sleep0_obesity1_pop_size + alc1_sleep0_obesity1_pop_size)/untreat_pop_size

uncertain1_rat = math.sqrt((causal_ratio_part1 * (1-causal_ratio_part1))/(treated_pop_size))
uncertain2_rat = math.sqrt((causal_ratio_part2 * (1-causal_ratio_part2))/(untreat_pop_size))
print("uncertain2", 1.96 * math.sqrt(uncertain1_rat + uncertain2_rat))


Causal Risk Difference: 0.054332
uncertain1 0.018657698829743982


Causal Risk Ratio: 1.075241
uncertain2 0.227338029780248
